# Get Started With Reasoning Models

This is your first introduction to working with reasoning models, code-first. In this notebook, we'll primarily use the o4-mini model. However, we will also explore the `o1` model briefly, to understand how the API works for both. Use [this table](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure%2Cpy#api--feature-support) for the latest information on supported features. And visit the [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python%2Cpy) documentation page for API details and code snippets. Here's a quick summary for convenience:

<br/>

| Characteristic | o1 | o4-mini |
|:--- |:---|:---|
| Developer Messages    | ✅ | ✅ |
| Structured Outputs    | ✅ | ✅ |
| Context Window Input  | 200K | 100K |
| Context Window Output | 200K | 100K |
| Reasoning Effort      | ✅ | ✅ |
| Vision Support        | ✅ | ✅ |
| Chat Completions API  | ✅ | ✅ |
| Responses API         | ✅ |    |
| Functions / Tools     | ✅ | ✅ |
| max_completion_tokens | ✅ |    |
| System messages       | ✅ | ✅ |
| Reasoning summary     | ✅ | |
| Streaming             | ✅ | |
| Model Card | [o4-mini](https://ai.azure.com/explore/models/o4-mini/version/2025-04-16/registry/azure-openai) | [o1](https://ai.azure.com/explore/models/o1/version/2024-12-17/registry/azure-openai)  |
| api_version | 2025-04-01-preview | 2025-03-01-preview |

---

## 1. Quickstart

In this section, we'll do a quick check to make sure we have the right dependencies installed. We'll also test both o1 and o4-mini models but we'll primarily use the `o4-mini` unless explicitly stated otherwise.

### 1.1 Install Python Dependencies

In [ ]:
# Install and Upgrade Pip
!pip install --upgrade pip --quiet

# Install Required Packages
!pip install -q openai azure-identity python-dotenv --quiet

# You may need to updated your OpenAI Python library
!pip install openai --upgrade --quiet


### 1.2 Check Env Variables

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    print("Missing env: AZURE_OPENAI_ENDPOINT")
elif not os.getenv("AZURE_OPENAI_KEY"):
    print("Missing env: AZURE_OPENAI_KEY")
else: 
    print("Azure OpenAI endpoint and key are set.")

### 1.3 Define Utility Functions

In [ ]:
# Print stats for the response
def pretty_print(response, response_time):
    """
    Prints the response details in a formatted manner.
    Args:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
        response_time (float): The time taken to get the response.
    """
    print(".........................")
    print(f"Response time: {response_time:.2f} seconds")
    print(f"Total Tokens: {response.usage.total_tokens}")
    print(f"Prompt Tokens: {response.usage.prompt_tokens}")
    print(f"Completion Tokens: {response.usage.completion_tokens}")
    print(f"Reasoning Tokens: {response.usage.completion_tokens_details.reasoning_tokens}")
    print(f"Output Tokens: {response.usage.total_tokens - response.usage.completion_tokens_details.reasoning_tokens}")
    print(f"\nResponse:\n {response.choices[0].message.content}")
    print(f".........................")


In [ ]:

from openai import AzureOpenAI
import time

# Chat with o1
def o1_chat(prompt="hi", reasoning_level="medium", developer_message="You are a helpful assistant"):
    """
    Sends a chat completion request to the Azure OpenAI API o1 model.
    Args:
        prompt (str): The input prompt to generate a response for.
        reasoning_level (str): The reasoning effort level ('low', 'medium', 'high').
        developer_message (str): The developer message to set the context for the assistant.
    Returns:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
    """
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version="2025-04-01-preview"
    )
    try:
        request_time = time.time()
        response = client.chat.completions.create(
            model="o1",
            messages=[
                {"role": "developer", "content": developer_message},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=5000,
            reasoning_effort=reasoning_level
        )
        response_time = time.time() - request_time
        pretty_print(response, response_time)
        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None


In [ ]:
# Chat with o4_mini
def o4mini_chat(prompt="hi", reasoning_level="medium", developer_message="You are a helpful assistant", response_format=None):
    """
    Sends a chat completion request to the Azure OpenAI API o4-mini model.
    Args:
        prompt (str): The input prompt to generate a response for.
        reasoning_level (str): The reasoning effort level ('low', 'medium', 'high').
        developer_message (str): The developer message to set the context for the assistant.
        response_format (BaseModel, optional): The expected structured output format for the response.
    Returns:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
    """
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version="2025-04-01-preview"
    )
    try:
        request_time = time.time()
        response = client.chat.completions.create(
            model="o4-mini",
            messages=[
                {"role": "developer", "content": developer_message},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=5000,
            reasoning_effort=reasoning_level,
            response_format=response_format
        )
        response_time = time.time() - request_time
        pretty_print(response, response_time)
        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None

## 2.Test Models

In [ ]:
# Test o1
response = o1_chat(
    prompt="How many p's in hippopotamus?",
    reasoning_level="medium",
    developer_message="You are a helpful assistant."
)

In [ ]:
# Test o4mini
response = o4mini_chat(
    prompt="How many p's in hippopotamus?",
    reasoning_level="low",
    developer_message="You are a helpful assistant."
)

---

## 2. Let's Prompt!

In [ ]:
# -------------------
# MATH EXAMPLE
# -------------------
response = o4mini_chat(
    prompt="A train travels at 60 mph. How long does it take to travel 90 miles?",
    reasoning_level="low",
    developer_message="You are a math tutor. Explain the solution"
)
print(response.choices[0].message.content)

In [ ]:
# -------------------
#  MATH REASONING
# -------------------
prompt = "Jane has twice as many apples as Tom. Together they have 18 apples. How many does each person have?"
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

In [ ]:
# -------------------
#  SCIENCE REASONING
# -------------------
prompt = "Why does a metal spoon feel colder than a wooden spoon when left in the same room?"
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

In [ ]:
# -------------------
#  MULTI-STEP PLANNING
# -------------------
prompt = "Design a basic weekly study schedule to learn Python in 6 weeks, assuming 1 hour per weekday."
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

In [ ]:
# --------------------------------
# CONSTRAINT-BASED SCHEDULING
# --------------------------------
prompt = "Three employees—Alex, Sam, and Riley—must each work one 4-hour shift today. Only Alex and Riley can work before noon, and Sam can’t work past 4 PM. Create a valid schedule."
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

In [ ]:
# -------------------
# TRY YOUR OWN
# -------------------

# Write your own prompt here
prompt = ""  

# Change these if you want to experiment
reasoning_level = "medium"
developer_message = "You are a helpful assistant."
response = o4mini_chat(prompt, reasoning_level, developer_message)
print(response.choices[0].message.content)